In [2]:
from pathlib import Path
import numpy as np

In [1]:
from PIL import Image
import tensorflow as tf

In [3]:
FILE_PATH = Path('data/raw/Flickr2K')
print(FILE_PATH)

data/raw/Flickr2K


In [4]:
HR_SIZE = 128
SCALE = 2
BATCH_SIZE = 8

In [5]:
files = []
files.extend(FILE_PATH.rglob('*.jpg'))
files.extend(FILE_PATH.rglob('*.png'))
files = sorted(files)

In [6]:
print (len(files))

2650


In [7]:
#print(files)

In [8]:
print(len(files))
print(files[:3])
print(type(files[0]))

2650
[PosixPath('data/raw/Flickr2K/000001.png'), PosixPath('data/raw/Flickr2K/000002.png'), PosixPath('data/raw/Flickr2K/000003.png')]
<class 'pathlib.PosixPath'>


In [9]:
np.random.seed(7)

In [10]:
np.random.shuffle(files)

In [11]:
print(files[:10])

[PosixPath('data/raw/Flickr2K/001583.png'), PosixPath('data/raw/Flickr2K/001605.png'), PosixPath('data/raw/Flickr2K/000279.png'), PosixPath('data/raw/Flickr2K/002041.png'), PosixPath('data/raw/Flickr2K/000187.png'), PosixPath('data/raw/Flickr2K/001694.png'), PosixPath('data/raw/Flickr2K/002460.png'), PosixPath('data/raw/Flickr2K/002077.png'), PosixPath('data/raw/Flickr2K/001197.png'), PosixPath('data/raw/Flickr2K/000340.png')]


In [12]:
train_files = files[:2120]

In [13]:
len(train_files)

2120

In [14]:
val_files = files[2120:2120+265]

In [15]:
len(val_files)

265

In [16]:
test_files = files[2120+265:]

In [17]:
len(test_files)

265

In [18]:
print(train_files[0],type(train_files[0]))

data/raw/Flickr2K/001583.png <class 'pathlib.PosixPath'>


In [19]:
dir(tf.io)

['FixedLenFeature',
 'FixedLenSequenceFeature',
 'RaggedFeature',
 'SparseFeature',
 'TFRecordOptions',
 'TFRecordWriter',
 'VarLenFeature',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'decode_and_crop_jpeg',
 'decode_base64',
 'decode_bmp',
 'decode_compressed',
 'decode_csv',
 'decode_gif',
 'decode_image',
 'decode_jpeg',
 'decode_json_example',
 'decode_png',
 'decode_proto',
 'decode_raw',
 'decode_webp',
 'deserialize_many_sparse',
 'encode_base64',
 'encode_jpeg',
 'encode_png',
 'encode_proto',
 'extract_jpeg_shape',
 'gfile',
 'is_jpeg',
 'match_filenames_once',
 'matching_files',
 'parse_example',
 'parse_sequence_example',
 'parse_single_example',
 'parse_single_sequence_example',
 'parse_tensor',
 'read_file',
 'serialize_many_sparse',
 'serialize_sparse',
 'serialize_tensor',
 'write_file',
 'write_graph']

In [36]:
def load_image(path):

    tf_read = tf.io.read_file(path)
    tf_img_decode = tf.image.decode_image(tf_read,channels = 3, expand_animations = False)
    tf_img_float = tf.image.convert_image_dtype(tf_img_decode, tf.float32)
    img.set_shape([None,None,3])

    return tf_img_float
    

In [37]:
print(type(train_files))
print(type(train_files[0]))
print(train_files[0])

<class 'list'>
<class 'pathlib.PosixPath'>
data/raw/Flickr2K/001583.png


In [38]:
print(type(files), type(files[0]), files[0])

<class 'list'> <class 'pathlib.PosixPath'> data/raw/Flickr2K/001583.png


In [39]:
img = load_image(str(train_files[0]))
print(img.shape)
print(img.dtype)
print(float(tf.reduce_min(img)), float(tf.reduce_max(img)))

(1356, 2040, 3)
<dtype: 'float32'>
0.0 1.0


In [40]:
def make_hr_patch(img,hr_size):
    shape = tf.shape(img)
    h = shape[0]
    w = shape[1]

    condition = tf.logical_or(h < hr_size, w< hr_size)
    
    def resize_img():
        new_h = tf.maximum(h,hr_size)
        new_w = tf.maximum(w,hr_size)
        
        new_img = tf.image.resize(img, size =[new_h,new_w], method = 'bilinear')
        return new_img
        
    def no_resize():
        return img

    img = tf.cond(condition,resize_img,no_resize)

    hr_patch = tf.image.random_crop(img, size=[hr_size,hr_size,3])
    return hr_patch

In [41]:
img = load_image(str(train_files[0]))
hr = make_hr_patch(img, 128)
print(hr.shape)
print(hr.dtype)
print(float(tf.reduce_min(hr)), float(tf.reduce_max(hr)))

(128, 128, 3)
<dtype: 'float32'>
0.0 1.0


In [53]:
def make_lr_patch(hr, scale):
    h = tf.shape(hr)[0]
    w = tf.shape(hr)[1]

    new_h = h // scale
    new_w = w// scale

    lr_small = tf.image.resize(hr,size = [new_h,new_w], method = 'bicubic')
    lr_small = tf.clip_by_value(lr_small, 0.0, 1.0)

    return lr_small

In [54]:
img = load_image(str(train_files[0]))
hr = make_hr_patch(img, 128)
lr = make_lr_patch(hr, 2)

print(hr.shape, lr.shape)
print(float(tf.reduce_max(lr)))

(128, 128, 3) (64, 64, 3)
1.0


In [65]:
def make_pair(path,hr_size,scale):
    img = load_image(path)
    hr = make_hr_patch(img, hr_size)
    lr = make_lr_patch(hr,scale)

    return lr,hr

In [67]:
#lr, hr = make_pair(str(train_files[0]),128,2)
#print(lr.shape, hr.shape)

In [68]:
#train_paths = [str(p) for p in train_files[:200]]

In [69]:
#ds = tf.data.Dataset.from_tensor_slices(train_paths)
#def exists_tf(path):
#    return tf.io.gfile.exists(path)

#ds = ds.map(make_pair, num_parallel_calls=tf.data.AUTOTUNE)

In [70]:
#print(next(iter(ds)))

In [71]:
#element = next(iter(ds))

In [72]:
#lr, hr = element
#print("LR:", lr.shape)
#print("HR:", hr.shape)

In [51]:
##IDEMO BREE

In [64]:
def make_sr_dataset(paths,hr_size,scale,batchs_size,training):
    shuffle_buffer = 1000

    ds = tf.data.Dataset.from_tensor_slices(paths)
    
    if training:
        ds = ds.shuffle(shuffle_buffer,reshuffle_each_iteration = True)

    ds= ds.map(lambda p: make_pair(p,hr_size,scale),num_parallel_calls = tf.data.AUTOTUNE)

    if training:
        ds = ds.batch(batchs_size,drop_remainder = True)
    else:
        ds = ds.batch(batchs_size,drop_remainder = False)

    ds = ds.prefetch(tf.data.AUTOTUNE)

        
    return ds
    

{'_input_dataset': <_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>,
 '_use_inter_op_parallelism': True,
 '_map_func': <tensorflow.python.data.ops.structured_function.StructuredFunctionWrapper at 0x1482f5d10>,
 '_deterministic': 'default',
 '_preserve_cardinality': True,
 '_num_parallel_calls': <tf.Tensor: shape=(), dtype=int64, numpy=-1>,
 '_use_unbounded_threadpool': False,
 '_name': None,
 '_variant_tensor_attr': <tf.Tensor: shape=(), dtype=variant, value=<ParallelMapDatasetV2Op::Dataset>>,
 '_graph_attr': <tensorflow.python.framework.ops.Graph at 0x1090a8e40>,
 '_options_attr': <tensorflow.python.data.ops.options.Options at 0x14c3ab6d0>}